# Анализируем структуру расходов субъектов федерации по госзаказу на основе отчётов об исполнении бюджетов

### Цель работы: 
* разобраться с работой с Excel файлами отчётов об исполнении бюджетов
* научиться извлекать сведения о структуре государственного бюджета
* визуализировать структуру государственного бюджета

### Источники данных 
Консолидированные бюджеты субъектов Российской Федерации и бюджетов территориальных государственных внебюджетных фондов в виде годовых отчётов с 2013 по 2020 годы

http://roskazna.ru/ispolnenie-byudzhetov/konsolidirovannye-byudzhety-subektov/

Готовый архив можно скачать по ссылке https://cdn.ruarxive.org/public/datacollect/roskazna.ru/regbudgets_2014_2020.zip

Архив необходимо распаковать в папку files 

Для работы потребуется установить библиотеки pandas, xlrd и tabulate используйте командную строку:

    pip install pandas
    pip install xlrd
    pip install tabulate


Импортируем необходимые библиотеки для работы

In [6]:
import sys, os
import xlrd
import json
import pprint
from IPython.display import HTML, display
import tabulate
import pandas as pd

Особенность данных отчётов об исполнении бюджета в том что они в файлах Excel и в разных форматах, меняющихся год от года. Для этого необходимо написать относительно универсальную функцию извлечения данных

In [7]:
def extract_report_data(year, filter_vr='', regname_sheet_id=1, regname_cell_coord=[4,3], summ_col_id=6):
    filepath = 'files/%s' % year
    files = os.listdir(filepath)
    table = []
    for name in files:
        if name[-4:] == '.XLS':
            wb = xlrd.open_workbook(filepath  + '/' + name)
            name = name.lower()
            sheet = wb.sheet_by_index(regname_sheet_id)
            reg = sheet.cell(regname_cell_coord[0], regname_cell_coord[1])
            datasheet = wb.sheet_by_index(regname_sheet_id+1)
            adict = {}
            data = {}
            total = 0
            print('Обработка %s, регион %s' % (name, reg.value))            
            for row in datasheet.get_rows():
                if total == 0:
                    if type(row[1].value) == type(''):
                        if row[1].value.isdigit() and row[1].value == '200':
                            total = int(float(str(row[summ_col_id].value).replace(' ', '').replace('\xa0', '').replace(',', '.')))
                            continue
                    else:
                        if row[1].value == 200:
                            total = int(row[summ_col_id].value)
                            continue
#                    if row[1].value == '200':
 #                   total = row[6].value


                if type(row[5].value) == type(''):
                    if row[5].value.isdigit():
                        vr = int(row[5].value)
                    else:
                        continue
                else:
                    vr = int(row[5].value)
                if vr >= 100:
                    if str(vr) not in adict.keys():
                        adict[str(vr)] = row[0].value
                    v = data.get(str(vr), 0)
                    data[str(vr)] = v + row[6].value if type(row[6].value) == float else 0
            for key, value in data.items():
                if filter_vr == '' or filter_vr == key:
                    table.append({'filename': name, 'region' : reg.value, 'vr' : key, 'vr_name' : adict[key], 'total' : total, 'vr_value' : int(value), 'vr_share' : (value * 100.0) / total})
    return table


In [8]:
table = extract_report_data('2016', '200', 1, [4,3], 6)


Обработка 0503317-3_fo_001-2582480.xls, регион Республика Башкортостан
Обработка 0503317-3_fo_002-2578483.xls, регион Республика Бурятия
Обработка 0503317-3_fo_003-2622844.xls, регион Республика Дагестан
Обработка 0503317-3_fo_004-2586781.xls, регион Кабардино-Балкарская Республика
Обработка 0503317-3_fo_005-2582358.xls, регион Республика Калмыкия
Обработка 0503317-3_fo_006-2595010.xls, регион Республика Карелия
Обработка 0503317-3_fo_007-2580361.xls, регион Республика Коми
Обработка 0503317-3_fo_008-2554128.xls, регион Республика Марий Эл
Обработка 0503317-3_fo_009-2547732.xls, регион Республика Мордовия
Обработка 0503317-3_fo_010-2577291.xls, регион Республика Северная Осетия-Алания
Обработка 0503317-3_fo_011-2590573.xls, регион Республика Татарстан (Татарстан)
Обработка 0503317-3_fo_012-2563872.xls, регион Республика Тыва
Обработка 0503317-3_fo_013-2546190.xls, регион Удмуртская Республика
Обработка 0503317-3_fo_014-2586589.xls, регион Республика Ингушетия
Обработка 0503317-3_fo_015

In [7]:
FIELDS = ['filename', 'region', 'vr',  'total', 'vr_value', 'vr_share']
df2016 = pd.DataFrame(table, columns=FIELDS)
df2016.sort_values(by=['vr_share'], ascending=True)

,filename,region,vr,total,vr_value,vr_share
11,0503317-3_fo_012-2563872.xls,Республика Тыва,200,30242840595,1864763452,6.165967
34,0503317-3_fo_035-2557940.xls,Калининградская область,200,100127031184,6399048015,6.390930
1,0503317-3_fo_002-2578483.xls,Республика Бурятия,200,70839571597,4883525984,6.893782
78,0503317-3_fo_080-2590348.xls,Республика Хакасия,200,45826134358,3454509505,7.538296
13,0503317-3_fo_014-2586589.xls,Республика Ингушетия,200,35446700854,2774528656,7.827326
...,...,...,...,...,...,...
73,0503317-3_fo_075-2597181.xls,Республика Крым,200,157007606940,25695498481,16.365767
40,0503317-3_fo_041-2597042.xls,Костромская область,200,39125358860,6630032561,16.945615
39,0503317-3_fo_040-2560260.xls,Кировская область,200,73019054301,12511662308,17.134791
44,0503317-3_fo_045-2635043.xls,Ленинградская область,200,160752950025,29134508288,18.123778


In [8]:

table2018 = extract_report_data('2018', '200', 0, [5,3], 6)


Обработка 01_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ БАШКОРТОСТАН
Обработка 02_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ БУРЯТИЯ
Обработка 03_01.01.2019_0503317.xls, регион Министерство финансов Республики Дагестан
Обработка 04_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ КАБАРДИНО- БАЛКАРСКОЙ РЕСПУБЛИКИ
Обработка 05_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ КАЛМЫКИЯ
Обработка 06_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ КАРЕЛИЯ
Обработка 07_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ КОМИ
Обработка 08_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ МАРИЙ ЭЛ
Обработка 09_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ МОРДОВИЯ
Обработка 10_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ СЕВЕРНАЯ ОСЕТИЯ-АЛАНИЯ
Обработка 11_01.01.2019_0503317.xls, регион МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ ТАТАРСТАН
Обработка 12_01.01.2019_0503

In [9]:
FIELDS = ['filename', 'region', 'vr',  'total', 'vr_value', 'vr_share']
df2018 = pd.DataFrame(table2018, columns=FIELDS)
df2018.sort_values(by=['vr_share'], ascending=False)

,filename,region,vr,total,vr_value,vr_share
16,17_01.01.2019_0503317.xls,МИНИСТЕРСТВО ФИНАНСОВ АЛТАЙСКОГО КРАЯ,200,152650051914,27055251889,17.723710
10,11_01.01.2019_0503317.xls,МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ ТАТАРСТАН,200,371881410298,64847943737,17.437802
25,26_01.01.2019_0503317.xls,ДЕПАРТАМЕНТ ФИНАНСОВ И БЮДЖЕТНОЙ ПОЛИТИКИ БЕЛГ...,200,125676672404,20946770465,16.667191
44,45_01.01.2019_0503317.xls,КОМИТЕТ ФИНАНСОВ ЛЕНИНГРАДСКОЙ ОБЛАСТИ,200,181666454636,30049717547,16.541148
35,36_01.01.2019_0503317.xls,МИНИСТЕРСТВО ФИНАНСОВ ТВЕРСКОЙ ОБЛАСТИ,200,87362912331,14176589289,16.227240
...,...,...,...,...,...,...
6,07_01.01.2019_0503317.xls,МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ КОМИ,200,107300781803,7682543022,7.159820
11,12_01.01.2019_0503317.xls,Министерство финансов Республики Тыва,200,38509516294,2703839990,7.021226
48,49_01.01.2019_0503317.xls,МИНИСТЕРСТВО ФИНАНСОВ МУРМАНСКОЙ ОБЛАСТИ,200,101655516711,7099959720,6.984333
81,88_01.01.2019_0503317.xls,"ДЕПАРТАМЕНТ ФИНАНСОВ, ЭКОНОМИКИ И ИМУЩЕСТВЕННЫ...",200,38334992700,2509179600,6.545403


Обратим внимание что в данных 2016 года из Excel файлов вылавливается наименование "Республика Башкортостан", а в данных 2018 года именование изменилось и теперь там указывается "МИНИСТЕРСТВО ФИНАНСОВ РЕСПУБЛИКИ БАШКОРТОСТАН".

Это означает что если мы хотим измерить изменения в доле закупок в Республике Башкортостан за несколько лет, то мы не можем просто сопоставлять по наименованию региона. Однаком мы можем увидеть что в 2016 году файл отчёта по Республике Башкортостан выглядел как "0503317-3_fo_001-2582480.xls", а в 2018 году "01_01.01.2019_0503317.xls". Структура названия файла отличается однако и там и там используется код "001" и "01" - это код региона. Давайте составим справочник кодов и переработаем функцию извлечения данных под получение кода из имени файла


In [3]:
def extract_reg_dict():
    year = '2020'
    filepath = 'files/%s' % year
    files = os.listdir(filepath)
    adict = {}
    for name in files:
        if name[-4:] == '.XLS':
            wb = xlrd.open_workbook(filepath  + '/' + name)
            name = name.lower()
            sheet = wb.sheet_by_index(0)
            reg = sheet.cell(6,3)
            regcode = name.split('_', 1)[0]#.rsplit('_', 1)[-1][1:]
            adict[regcode] = reg.value
    return adict

In [4]:
regcodes = extract_reg_dict()
regcodes.items()

dict_items([('01', 'Бюджет Республики Башкортостан'), ('02', 'РЕСПУБЛИКАНСКИЙ БЮДЖЕТ'), ('03', 'Республиканский бюджет Республики Дагестан'), ('04', 'Республиканский бюджет Кабардино-Балкарской Республики'), ('05', 'Республиканский бюджет'), ('06', 'Бюджет Республики Карелия'), ('07', 'республиканский бюджет Республики Коми'), ('08', 'Республиканский бюджет Республики Марий Эл'), ('09', 'Республиканский бюджет Республики Мордовия'), ('10', 'Республиканский бюджет Республики Северная Осетия-Алания'), ('11', 'бюджет Республики Татарстан'), ('12', 'Республиканский бюджет Республики Тыва'), ('13', 'Бюджет Удмуртской Республики'), ('14', 'Республиканский бюджет'), ('15', 'Республиканский бюджет Чувашской Республики'), ('16', 'Государственный бюджет Республики Саха (Якутия)'), ('17', 'Краевой бюджет'), ('18', 'Бюджет Краснодарского края'), ('19', 'бюджет Красноярского края'), ('20', 'Бюджет Приморского края'), ('21', 'Бюджет Ставропольского края'), ('22', 'Бюджет Хабаровского края'), ('23', 

Перепишем функцию извлечения данных таким образом чтобы мы могли получать код региона. В параметр ns2016 указываем True если год до 2016 включительно и False если 2017 и 2018 годы. В зависимости от этого разбираем имя файла

In [12]:
def adv_extract_data(year, filter_vr='', regname_sheet_id=1, regname_cell_coord=[4,3], summ_col_id=6, ns2016=True):
    print('Обрабатываем данные за %s' % (year))
    filepath = 'files/%s' % year
    files = os.listdir(filepath)
    table = []
    for name in files:
        if name[-4:] == '.XLS':
            regcode = name.rsplit('-', 1)[0].rsplit('_', 1)[-1][1:] if ns2016 else name.split('_', 1)[0]
            if not regcode.isdigit(): continue
            wb = xlrd.open_workbook(filepath  + '/' + name)
            name = name.lower()
            datasheet = wb.sheet_by_index(regname_sheet_id+1)
            adict = {}
            data = {}
            total = 0
            for row in datasheet.get_rows():
                if total == 0:
                    if type(row[1].value) == type(''):
                        if row[1].value.isdigit() and row[1].value == '200':
                            total = int(float(str(row[summ_col_id].value).replace(' ', '').replace('\xa0', '').replace(',', '.')))
                            continue
                    else:
                        if row[1].value == 200:
                            total = int(row[summ_col_id].value)
                            continue

                if type(row[5].value) == type(''):
                    if row[5].value.isdigit():
                        vr = int(row[5].value)
                    else:
                        continue
                else:
                    vr = int(row[5].value)
                if vr >= 100:
                    if str(vr) not in adict.keys():
                        adict[str(vr)] = row[0].value
                    v = data.get(str(vr), 0)
                    data[str(vr)] = v + row[summ_col_id].value if type(row[summ_col_id].value) == float else 0
            for key, value in data.items():
                if filter_vr == '' or filter_vr == key:
                    table.append({'year' : int(year), 'filename': name, 'regcode' : regcode, 'region' : regcodes[regcode], 'vr' : key, 'vr_name' : adict[key], 'total' : total, 'vr_value' : int(value), 'vr_share' : (value * 100.0) / total})
    return table


А теперь подсчитаем данные за 2015-2020 годы и сведем их в одну таблицу

In [15]:
table = []
#table.extend(adv_extract_data('2013', '200', 1, [4,3], 7, True))
#table.extend(adv_extract_data('2014', '200', 1, [4,3], 7, True))
table.extend(adv_extract_data('2015', '200', 1, [4,3], 7, True))
table.extend(adv_extract_data('2016', '200', 1, [4,3], 6, True))
table.extend(adv_extract_data('2017', '200', 0, [5,3], 6, False))
table.extend(adv_extract_data('2018', '200', 0, [5,3], 6, False))
table.extend(adv_extract_data('2019', '200', 0, [5,3], 6, False))
table.extend(adv_extract_data('2020', '200', 0, [5,3], 6, False))

Обрабатываем данные за 2015
Обрабатываем данные за 2016
Обрабатываем данные за 2017
Обрабатываем данные за 2018
Обрабатываем данные за 2019
Обрабатываем данные за 2020


После чего преобразуем их в один Pandas datafame и посмотрим что получается на выходе. Заодно сделаем выборку по выбранному региону: "Ярославская область" (код 71) и отсортируем по годам.

In [16]:
FIELDS = ['year', 'regcode', 'region', 'vr',  'total', 'vr_value', 'vr_share']
dfall = pd.DataFrame(table, columns=FIELDS)
df_yar= dfall.loc[dfall['regcode'] == '60']
df_yar.sort_values(by=['regcode', 'year'], ascending=True)

,year,regcode,region,vr,total,vr_value,vr_share
59,2015,60,Областной бюджет,200,121686539335,9112647991,7.488624
145,2016,60,Областной бюджет,200,120802848177,9516899728,7.878043
231,2017,60,Областной бюджет,200,125808686837,12340575467,9.809001
317,2018,60,Областной бюджет,200,135533123127,13402811290,9.888956
403,2019,60,Областной бюджет,200,156850717294,21703626912,13.837123
490,2020,60,Областной бюджет,200,188112083888,22853185942,12.148707


А теперь попробуем посмотреть на структуру расходов одного региона по видам расходов. В этот раз не будем фильтровать по виду расходов и получим данные по всем

In [17]:
table_full = []
#table.extend(adv_extract_data('2013', '200', 1, [4,3], 7, True))
#table.extend(adv_extract_data('2014', '200', 1, [4,3], 7, True))
table_full.extend(adv_extract_data('2015', '', 1, [4,3], 7, True))
table_full.extend(adv_extract_data('2016', '', 1, [4,3], 6, True))
table_full.extend(adv_extract_data('2017', '', 0, [5,3], 6, False))
table_full.extend(adv_extract_data('2018', '', 0, [5,3], 6, False))
table_full.extend(adv_extract_data('2019', '', 0, [5,3], 6, False))
table_full.extend(adv_extract_data('2020', '', 0, [5,3], 6, False))


Обрабатываем данные за 2015
Обрабатываем данные за 2016
Обрабатываем данные за 2017
Обрабатываем данные за 2018
Обрабатываем данные за 2019
Обрабатываем данные за 2020


А теперь подсчитаем структуру расходов бюджета г.Санкт-Петербурга за 2018 год воспользовавшись фильтрами в Dataframe

In [18]:
FIELDS = ['year', 'regcode', 'region', 'vr', 'vr_name', 'total', 'vr_value', 'vr_share']
dffull = pd.DataFrame(table_full, columns=FIELDS)

In [19]:
dffull.loc[dffull['regcode'] == '77'].loc[dffull['vr'].isin(['100', '200', '300', '400', '500', '600', '700', '800', '900'])].loc[dffull['year'] == 2019]


,year,regcode,region,vr,vr_name,total,vr_value,vr_share
34085,2019,71,Бюджет Ярославской области,100,Расходы на выплаты персоналу в целях обеспечен...,103556427591,7289403978,7.039065
34090,2019,71,Бюджет Ярославской области,200,"Закупка товаров, работ и услуг для обеспечения...",103556427591,12252327979,11.831548
34095,2019,71,Бюджет Ярославской области,500,Межбюджетные трансферты,103556427591,0,0.000000
34097,2019,71,Бюджет Ярославской области,800,Иные бюджетные ассигнования,103556427591,5517172135,5.327696
34105,2019,71,Бюджет Ярославской области,300,Социальное обеспечение и иные выплаты населению,103556427591,30181402871,29.144886
34124,2019,71,Бюджет Ярославской области,400,Капитальные вложения в объекты государственной...,103556427591,5047658829,4.874308
34131,2019,71,Бюджет Ярославской области,600,"Предоставление субсидий бюджетным, автономным ...",103556427591,39477260902,38.121497
34165,2019,71,Бюджет Ярославской области,700,Обслуживание государственного (муниципального)...,103556427591,2845445868,2.747725


In [20]:
df_reg = dffull.loc[dffull['regcode'] == '41'].loc[dffull['vr'].isin(['100', '200', '300', '400', '500', '600', '700', '800', '900'])]
df_reg.sort_values(by=['regcode', 'year', 'vr'], ascending=True)

,year,regcode,region,vr,vr_name,total,vr_value,vr_share
3057,2015,41,Областной бюджет Костромской области,100,Расходы на выплаты персоналу в целях обеспечен...,36411363409,6298158493,17.297233
3065,2015,41,Областной бюджет Костромской области,200,"Закупка товаров, работ и услуг для государстве...",36411363409,5142362294,14.122960
3068,2015,41,Областной бюджет Костромской области,300,Социальное обеспечение и иные выплаты населению,36411363409,11699468857,32.131367
3094,2015,41,Областной бюджет Костромской области,400,Капитальные вложения в объекты государственной...,36411363409,1991342914,5.469015
3078,2015,41,Областной бюджет Костромской области,500,Межбюджетные трансферты,36411363409,323206952,0.887654
3103,2015,41,Областной бюджет Костромской области,600,"Предоставление субсидий бюджетным, автономным ...",36411363409,7623549979,20.937282
3108,2015,41,Областной бюджет Костромской области,700,Обслуживание государственного (муниципального)...,36411363409,1323087827,3.633722
3062,2015,41,Областной бюджет Костромской области,800,Иные бюджетные ассигнования,36411363409,2010186089,5.520766
9622,2016,41,Областной бюджет Костромской области,100,Расходы на выплаты персоналу в целях обеспечен...,39125358860,6504816202,16.625576
9627,2016,41,Областной бюджет Костромской области,200,"Закупка товаров, работ и услуг для обеспечения...",39125358860,6630032561,16.945615


In [24]:
df_reg = dffull.loc[dffull['regcode'] == '21'].loc[dffull['vr'].isin(['630'])]
df_reg.sort_values(by=['regcode', 'year', 'vr'], ascending=True)

,year,regcode,region,vr,vr_name,total,vr_value,vr_share
1555,2015,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,132245162766,2383194997,1.802104
8078,2016,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,133433216780,764611914,0.573030
14922,2017,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,145800959465,987927620,0.677587
22250,2018,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,156053995137,1031007371,0.660673
29716,2019,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,180579799197,1840301238,1.019107
37382,2020,21,Бюджет Ставропольского края,630,Субсидии некоммерческим организациям (за исклю...,211595875261,1840778072,0.869950


Вопросы, сообщения об ошибках и пожелания направляйте на ibegtin@infoculture.ru

Подсчитаем объём расходов на некоммерческие организации через субсидии регионального бюджета. Это 630 код вида расходов.

In [5]:
df_reg = dffull.loc[dffull['year'] == 2020].loc[dffull['vr'].isin(['630'])]
df_reg.sort_values(by=['regcode', 'year', 'vr'], ascending=True)

NameError: name 'dffull' is not defined

In [55]:
ngo2020 = dffull.loc[dffull['year'] == 2020].loc[dffull['vr'].isin(['630'])].sum(axis=0)['vr_value'] / 1000000000
ngo2020

779.442103041

In [56]:
ngo2019 = dffull.loc[dffull['year'] == 2019].loc[dffull['vr'].isin(['630'])].sum(axis=0)['vr_value'] / 1000000000
ngo2019

725.246200771

In [57]:
ngo2018 = dffull.loc[dffull['year'] == 2018].loc[dffull['vr'].isin(['630'])].sum(axis=0)['vr_value'] / 1000000000
ngo2018

345.562086745